In [1]:
import pandas as pd
import numpy as np
from time import time

from scipy.spatial.distance import cosine

In [2]:
path = '../data/'
path_items = '../data/KDD2023_embeddings/'

filename = 'prueba_preds.csv'
items_filename = 'Primeros_embeddings.csv'


t = time()
chunk = pd.read_csv(path+filename,chunksize=1e5)
print(f"Tiempo chunck1:\t{time()-t}")
test_df = pd.concat(chunk)
print(f"Tiempo total1:\t{(time()-t)/60} min\n")


t = time()
#chunk2 = pd.read_csv('C:/Users/usuario/MasterIM/TFM/data/KDD2023_embeddings/'+items_filename,chunksize=1e5)
chunk2 = pd.read_csv(path_items+items_filename,chunksize=1e5)
print(f"Tiempo chunck2:\t{time()-t}")
items_df = pd.concat(chunk2)
print(f"Tiempo total2:\t{(time()-t)/60} min\n")

Tiempo chunck1:	0.0
Tiempo total1:	0.00013329585393269858 min

Tiempo chunck2:	0.008005142211914062
Tiempo total2:	0.18332080841064452 min



In [3]:
items_df.head()

,ItemId,movie_id,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,0007440847,0,0.705512,-0.132848,0.360967,0.204053,-0.491573,0.525456,-1.061789,-0.450654,...,-0.676208,0.531552,-0.686487,0.003330,-0.311290,0.755625,0.135033,0.538100,0.209097,-0.048732
1,0007477155,1,-0.548283,0.000355,-0.857576,-0.516308,-0.164168,0.305688,0.822554,-0.304165,...,0.833657,0.844858,-0.135204,-0.634501,-0.174124,0.235411,0.106691,-0.248621,-0.488445,0.233837
2,0008260184,10,-0.648677,-0.475611,-0.211740,-0.030497,-0.084687,-0.893777,0.549404,-0.272313,...,0.181996,-0.526775,0.429677,0.377227,-0.394160,-0.711318,-0.110256,-0.051346,0.403998,0.276496
3,0441569595,100,-0.809459,0.160266,-0.462641,-1.598935,-0.229119,0.427275,0.659189,-0.290117,...,-0.787689,0.261570,0.718889,-0.159298,0.680047,-0.958712,-0.172936,-0.168796,0.013271,0.444924
4,1973363178,1000,0.711795,-0.133876,0.279420,-0.697728,-0.295069,-0.025046,-0.499776,-0.012979,...,-0.289264,0.664851,-0.508413,-0.638434,0.007214,0.349008,0.156555,-0.019364,0.082528,0.762302


In [4]:
items_df = items_df.set_index('ItemId').drop('movie_id',axis=1)
items_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
ItemId,,,,,,,,,,,,,,,,,,,,,
0007440847,0.705512,-0.132848,0.360967,0.204053,-0.491573,0.525456,-1.061789,-0.450654,-0.228735,-0.271682,...,-0.676208,0.531552,-0.686487,0.003330,-0.311290,0.755625,0.135033,0.538100,0.209097,-0.048732
0007477155,-0.548283,0.000355,-0.857576,-0.516308,-0.164168,0.305688,0.822554,-0.304165,0.544786,-0.223038,...,0.833657,0.844858,-0.135204,-0.634501,-0.174124,0.235411,0.106691,-0.248621,-0.488445,0.233837
0008260184,-0.648677,-0.475611,-0.211740,-0.030497,-0.084687,-0.893777,0.549404,-0.272313,1.324682,0.287618,...,0.181996,-0.526775,0.429677,0.377227,-0.394160,-0.711318,-0.110256,-0.051346,0.403998,0.276496
0441569595,-0.809459,0.160266,-0.462641,-1.598935,-0.229119,0.427275,0.659189,-0.290117,-0.139079,0.033957,...,-0.787689,0.261570,0.718889,-0.159298,0.680047,-0.958712,-0.172936,-0.168796,0.013271,0.444924
1973363178,0.711795,-0.133876,0.279420,-0.697728,-0.295069,-0.025046,-0.499776,-0.012979,1.036865,0.655368,...,-0.289264,0.664851,-0.508413,-0.638434,0.007214,0.349008,0.156555,-0.019364,0.082528,0.762302


In [5]:
test_df.head()

,next_item_prediction,next_item
0,"['B099DQ7GXV', 'B09N6RF3LY', 'B082TVG5FK', 'B0...",B09FKD61R8
1,"['B07QKFWZP8', 'B07RJX6B2N', 'B07JHPYHBK', 'B0...",B0749TX4YP
2,"['B095CXMGWG', '3423340002', 'B09YTTF232', 'B0...",B09B2WGPRB
3,"['B004LPAJSS', 'B08B43NGH3', 'B09LRPSRN6', 'B0...",B09B31WTVY
4,"['B0B4NLG264', 'B08ZL46HX3', 'B075L8DF6K', 'B0...",B000OXME9S


In [6]:
def str2list(x):
    x = x.replace('[', '').replace(']', '').replace("'", '').replace('\n', ' ').replace('\r', ' ').replace(',','')
    l = [i for i in x.split() if i]
    return l

In [7]:
test_df['next_item_prediction'] = test_df['next_item_prediction'].apply(str2list)

In [8]:
test_df.head()

,next_item_prediction,next_item
0,"[B099DQ7GXV, B09N6RF3LY, B082TVG5FK, B00RCWSSR...",B09FKD61R8
1,"[B07QKFWZP8, B07RJX6B2N, B07JHPYHBK, B07VJ7FRR...",B0749TX4YP
2,"[B095CXMGWG, 3423340002, B09YTTF232, B09JSGV16...",B09B2WGPRB
3,"[B004LPAJSS, B08B43NGH3, B09LRPSRN6, B013GBHCX...",B09B31WTVY
4,"[B0B4NLG264, B08ZL46HX3, B075L8DF6K, B09Q8357D...",B000OXME9S


In [9]:
# los v_* son los vectores de los eembeddings

def reord_zip(row):
    item = row['next_item']
    preds = row['next_item_prediction']
    v_item = items_df.loc[item]
    distances = [cosine(v_item,items_df.loc[p]) for p in preds]
    combined = list(zip(preds, distances))
    sorted_combined = sorted(combined, key=lambda x: x[1])
    sorted_items = [pair[0] for pair in sorted_combined]
    return sorted_items

def reord_argsort(row):
    item = row['next_item']
    preds = row['next_item_prediction']
    v_item = items_df.loc[item]
    distances = [cosine(v_item,items_df.loc[p]) for p in preds]
    sorted_indices = np.argsort(distances)
    sorted_items = [preds[i] for i in sorted_indices]
    return sorted_items


In [10]:
t = time()
test_df['nip_reord_zip'] = test_df.apply(reord_zip,axis=1)
print(f"Tiempo reordena zip:\t{time()-t}")

t = time()
test_df['nip_reord_args'] = test_df.apply(reord_argsort,axis=1)
print(f"Tiempo reordena args:\t{time()-t}")

Tiempo reordena zip:	0.49680304527282715
Tiempo reordena args:	0.27229952812194824


In [11]:
# Comprobamos que las ordenaciones son iguales
test_df.apply(lambda x: not x['nip_reord_zip']==x['nip_reord_args'],axis=1).sum()

0